# data.tensorf

> Fill in a module description here


In [ ]:
#| default_exp data.tensorf

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import numpy as np
import os
import sys
import random
import torch
import torchvision
from sklearn.model_selection import train_test_split
import ujson
import torchvision.transforms as transforms
from fastcore.utils import *
from fedai.utils import *

In [ ]:
#| export
random.seed(42)
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
#| export
class TensorF(torch.utils.data.Dataset):
    def __init__(self, cfg, client_idx, train= True):
        self.cfg = cfg
        self.config_path = self.cfg.data.dir_path + "config.json"
        self.train_path = self.cfg.data.dir_path + "train/"
        self.test_path = self.cfg.data.dir_path + "test/"
        self.train = train
        self.data_dict = self.load_single_client_data(client_idx)

    def tensorify(self, data):
        X = torch.Tensor(data['x']).type(torch.float32)
        y = torch.Tensor(data['y']).type(torch.int64)
        return {'x': X, 'y': y}

    def load_single_client_data(self, idx):
        data_dir = os.path.join(self.train_path) if self.train else os.path.join(self.test_path)

        data_file = os.path.join(data_dir, str(idx) + '.npz')
        with open(data_file, 'rb') as f:
            data = np.load(f, allow_pickle= True)['data'].tolist()
        
        data_dict = self.tensorify(data)

        return data_dict

    def __getitem__(self, index):
        return self.data_dict['x'][index], self.data_dict['y'][index]

    def __len__(self):
        return len(self.data_dict['x'])

In [ ]:
#| hide
path = '/home/ahmed/Ahmed-home/1- Projects/Research/publications/2024/letter 1/code/PFLlib/dataset/Cifar10/'
from omegaconf import DictConfig, OmegaConf
import argparse
import os
args = dict()
args['data']= dict()
args['data']['dir_path'] = path
args = argparse.Namespace(**args)

cfg = OmegaConf.create(vars(args))


In [ ]:
#| hide
train_ds = TensorF(cfg, 0, train=True)
train_ds[0][0].shape, train_ds[0][1].item()


(torch.Size([3, 32, 32]), 9)

In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()